# Abstract

# Background

# Mathematical Representation

Probably a mostly analytical/markdowny section with a bunch of latex laying out the math

### Cost Functional/Constraints

### Pontryagin's Maximum Principle 

### Gradient

# Solution

In [ ]:
#Import stuff

### Method of Characteristics

### Ian's Code

### Pyclaw

# Interpretation

# Conclusion